# Robot With Motion Planner

The robot with motion planner slightly extends the robot interface presented in the first tutorial, by adding motion planning capabilities based on the motion planner introduced in tutorial 2. This is basically a combination of them with few more features. To create an instance, a motion planner should be provided. Note that it can be an extension of the motion planner that you can implement with different geometry as shown in tutorial 2. The controller is agnostic to your geometry and motion planner implementation, as long as it implements the abstract motion planner interface.

In [ ]:
from ur_lab.manipulation.robot_with_motion_planning import RobotInterfaceWithMP
from ur_lab.robot_inteface.robots_metadata import ur5e_1

# from previous tutorial:
from ur_lab.motion_planning.motion_planner import MotionPlanner
from ur_lab.motion_planning.geometry_and_transforms import GeometryAndTransforms

mp = MotionPlanner()
gt = GeometryAndTransforms.from_motion_planner(mp)
robot = RobotInterfaceWithMP(ur5e_1["ip"], ur5e_1["name"], mp, gt)

# shortcut for convenience:
# robot = RobotInterfaceWithMP.build_from_robot_name_and_ip(ur5e_1["ip"], ur5e_1["name"])

plan_and_moveJ method moves to a specified config. If there is no direct path available it already handles the motion planning for you. Note that there is a plan_and_move_home method that you can use for better safety than move_home.

In [ ]:
q1 = [0.03923, -1.76654, -2.27161, -2.19254, 0.40798, 3.07572]
q2 = [-0.06798, -1.84265, -2.65451, 1.33179, 1.61574, 1.64959]

robot.plan_and_move_home()
robot.plan_and_moveJ(q1)

# No direct path available, motion planning is needed:
robot.plan_and_moveJ(q2)

This interface also has another method that may be helpful, plan_and_move_to_xyzrz will move the end effector to the desired cartesian coordinates with the gripper facing down, rotated around z axis by rz. This is useful for some pick and place tasks. It can be be followed by a linear movement downward, or even move_until_contact to pick an object.

In [ ]:
robot.plan_and_move_home()

import numpy as np
robot.plan_and_move_to_xyzrz(x=0.4, y=0.0, z=0.2, rz=0)
robot.plan_and_move_to_xyzrz(x=0.5, y=-0.2, z=0.2, rz=-np.pi/2)
robot.plan_and_move_to_xyzrz(x=0.5, y=-0.2, z=0.2, rz=-3*np.pi/4)

``Note: Similarely to the motion planner, in such non-native interfaces x, y, z are in world coordinates and not in robot base coordinates. The origin of the world coordinates is the coordinates in the klampt world file that the motion planner uses``

# Manipulation Interface
Another extension for the robot interface is the manipulation interface. It extends the robot interface with gripper control and pick and place capabilities. It is built on top of the robot with motion planning interface, so it has all the capabilities of the previous interface.

The inheritance hierarchy can be viewed in the following diagram:


<img src="assets/robots_arch.png" width="700"/>

Note that the grasping methods will do nothing unless there is a gripper installed on the robot. Here we show the 2FG gripper interface. Note that there is another interface for the Vacuum gripper which is similar but slightly different.
There is a bit more than what we show here, feel free to view the implementation and the interface.

In [ ]:
del robot # we are going to open a new connection to the robot through different interface. Need to close this one.

The 2FG manipulation controller inherits the robot with motion planning interface, so it has all the capabilities of the previous interface:

In [ ]:
from ur_lab.manipulation.manipulation_controller_2fg import ManipulationController2FG
from ur_lab.robot_inteface.robots_metadata import ur5e_1
import numpy as np

robot = ManipulationController2FG.build_from_robot_name_and_ip(ur5e_1["ip"], ur5e_1["name"])
robot.plan_and_move_home()

robot.plan_and_moveJ([0, -np.pi/2, -np.pi/2, -np.pi/2, np.pi/2, 0])

Gripper control:

In [ ]:
robot.grasp()

In [ ]:
robot.release_grasp()

Note that there are arguments to control force, speed, and width. There is also wait time for the program to block the next action. This is useful to prevent the next movement action after grasp from starting before the gripper is fully closed.

In [ ]:
robot.grasp(wait_time=0.2, force=2, speed=10, width=60)

## Generalized Pick and Place

In [ ]:
point = [0.4, -0.4, 0.2]
# come from above:
robot.pick_up_at_angle(point=point, start_offset=[0, 0, 0.1], ee_rz=np.pi)
# come from 45 degrees above and the negative y direction:
robot.pick_up_at_angle(point=point, start_offset=[0, 0.1, 0.1], ee_rz=np.pi)
# same but rotate gripper
robot.pick_up_at_angle(point=point, start_offset=[0, 0.1, 0.1], ee_rz=0)
# same but start linear movement from further away
robot.pick_up_at_angle(point=point, start_offset=[0, 0.2, 0.2], ee_rz=0)

In [ ]:
# put down from the side:
robot.put_down_at_angle(point=point, start_offset=[-0.1, -0.05, 0], ee_rz=0)

In [ ]:
robot.plan_and_move_home()

==Point is in world coordinates!==